# Sequential Agent

This example shows how to create a simple sequential agent to perform a chain of subtasks by agents.

In [1]:
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from adp_core.agent import BaseAgent
from adp_core.chain import BaseLLMChain
from adp_core.orchestration import SequentialAgent
from adp_core.types import AgentMessage
from adp_langchain.chain import ChatCausalMultiTurnsChain
from langchain_ollama import ChatOllama

## Define the agent

Create a simple agent for demonstration

In [2]:
from pydantic import Field


class FunctionImplementationAgent(BaseAgent):
    chain: BaseLLMChain = Field(...)

    def execute(self, message: AgentMessage, **kwargs):
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        self.state = "idle"
        message.execution_result = "success"
        message.origin = self.card.name
        return message

In [3]:
model_code = ChatOllama(model="codegemma:2b-v1.1-q8_0-16384", base_url="192.168.55.1:11434")
model_doc = ChatOllama(model="mistral:7b-instruct-v0.3-q4_K_S", base_url="192.168.55.1:11434")
model_review = ChatOllama(model="qwen3:4b-thinking-2507-q4_K_M", base_url="192.168.55.1:11434")

In [4]:
code_system_prompt = """You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Just the implementation, DO NOT comment anything else"""
code_user_prompt = "{query}"
code_chain = ChatCausalMultiTurnsChain(model_code, code_system_prompt, code_user_prompt)
code_chain.final_repsonse_as_context("code")

doc_system_prompt = """You are a great developer avocate.
You task is to write a document and in-code explain for the function.
Just the comment and explanation, DO NOT change the logic of the provided function.
The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code"""
doc_user_prompt = """Input query: {query}

Function implementation: {context_code}
"""
doc_chain = ChatCausalMultiTurnsChain(model_doc, doc_system_prompt, doc_user_prompt)
doc_chain.final_repsonse_as_context("doc")

review_system_prompt = """You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain code, inline code comment and docstring, nothing else."""
review_user_prompt = """Input query: {query}

Function implementation: {context_doc}
"""
review_chain = ChatCausalMultiTurnsChain(model_review, review_system_prompt, review_user_prompt)
review_chain.final_repsonse_as_context("review")

def state_callback(state: str):
    print(state)
code_skill = AgentSkill(
    id='code-skill',
    name="code skill",
    description="self-code skill",
    tags=['code']
)
code_card = AgentCard(
    name="code agent",
    description="self-code agent",
    skills=[code_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
doc_skill = AgentSkill(
    id='doc-skill',
    name="doc skill",
    description="self-doc skill",
    tags=['doc']
)
doc_card = AgentCard(
    name="doc agent",
    description="self-doc agent",
    skills=[doc_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
review_skill = AgentSkill(
    id='review-skill',
    name="review skill",
    description="self-review skill",
    tags=['review']
)
review_card = AgentCard(
    name="review agent",
    description="self-review agent",
    skills=[review_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
sequential_skill = AgentSkill(
    id='sequential-skill',
    name="sequential skill",
    description="self-sequential skill",
    tags=['sequential']
)
sequential_card = AgentCard(
    name="sequential agent",
    description="self-sequential agent",
    skills=[sequential_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
code_agent = FunctionImplementationAgent(chain=code_chain, card=code_card, state_change_callback=state_callback)
doc_agent = FunctionImplementationAgent(chain=doc_chain, card=doc_card, state_change_callback=state_callback)
review_agent = FunctionImplementationAgent(chain=review_chain, card=review_card, state_change_callback=state_callback)
agent = SequentialAgent(agents=[code_agent, doc_agent, review_agent], card=sequential_card, state_change_callback=state_callback)

In [5]:
query = "Write a python function to find the biggest but not exceed the given integer number. The found number must be a number in Fibonacci array."
message = agent.execute(AgentMessage(query=query))

for response in message.responses:
    name, msg = response
    print(name)
    print(msg)

sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:running)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:running)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:running))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:idle/((code agent:idle)-(doc agent:idle)-(review agent:idle))
code agent
py

doc agent
  Here is a Python function that finds the largest Fibonacci number less than or equal to the given integer `n`. This function uses dynamic programming (memoization) to efficiently compute Fibonacci numbers:

```python
def fib_largest(n):
    fib = [0, 1]
    max_fib = -1
    for i in range(2, n+1):
        next